# Data Preprocessing Tools

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2024-11-23 12:35:19.591394: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-23 12:35:19.605013: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732332919.615929   28385 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732332919.618992   28385 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-23 12:35:19.629786: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## Part 1 - Data Preprocessing

### Importing dataset

data를 정제하기 쉬운 방법을 택하면 됨, 보통 컬럼 이름이 있는 데이터가 많다보니 pandas를 이용하는 편이 좋음

In [2]:
dataset = pd.read_csv('../Data/Part 1 - Artificial Neural Networks (ANN)/Churn_Modelling.csv')
df = pd.DataFrame(dataset)
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


머신러닝에 필요하지 않은 데이터는 제거, 처음 3개의 컬럼은 필요하지 않을것으로 보임

In [3]:
X = df.iloc[:, 3:-1].values
y = df.iloc[:, -1].values
print(X,y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]] [1 0 1 ... 1 1 0]


---

### Encoding categorical data

##### 성별 컬럼 encoding 하기
머신러닝은 숫자 데이터만 처리할 수 있으므로, gender를 0,1 로 바꾸어 줌 \
0 / 1 은 구분을 위한 숫자일 뿐 `크기`를 갖고 있지 않다는 점을 확인

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [5]:
print(X)  # gender 순서는 알파벳 순서를 따름. 다르게 mapping 가능

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


##### One Hot Encoding -- 'Geography column'

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [7]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


##### Training / Test set으로 분리

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

##### Feature Scaling

###### Machine Learning 알고리즘은 `데이터의 크기`에 민감하기 때문
- age: 20 ~ 70
- salery: 10,000 ~ 100,1000 (큰 값)
###### 스케일 차이가 큰 값을 학습하면 모델이 큰 값에 더 많은 가중치를 부여하기 때문 == 여러가지 방법이 있으나, 나중에 학습

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

---

## Part 2 - Building ANN
#### Initializing the ANN

In [13]:
ann = tf.keras.models.Sequential()

#### Adding input layer and first hidden layer

In [14]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

##### Adding second hidden layer

In [15]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

##### Adding OUTPUT layer

In [17]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))